In [ ]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

def show_tables(conn):
    """ show all tables in the database """
    cur = conn.cursor()
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(query)
    result = cur.fetchall()
    return [result[i][0] for i in range(len(result))]

def drop_temporary_tables(conn):
    """ drop temporary tables in the database """
    cur = conn.cursor()
    temporary_table_names = [table for table in show_tables(conn) if 'temporary' in table]
    for temporary_table_name in temporary_table_names:
        cur.execute(f"DROP TABLE IF EXISTS {temporary_table_name};")
    conn.commit()
    
def print_query(query):
    cur = conn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    for row in result:
        print(row)
        
zdb_path = f'EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb'

try:
    conn = sqlite3.connect(zdb_path)
except Error as e:
    print(e)
# TODO: to do this, we need to get start time from pnmlog or pnmexp
# get recording start stop
cur = conn.cursor()
query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
cur.execute(query)
result = cur.fetchall()
recording_start = int(result[0][0])
query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
cur.execute(query)
result = cur.fetchall()
recording_stop = int(result[0][0])
length_ns = recording_stop - recording_start # ns
length_s = length_ns * 1e-7 # s
hh = length_s // 3600
mm = (length_s % 3600) // 60
ss = ((length_s % 3600) % 60)
print(hh,mm,ss,length_s)
print(recording_start)
print(recording_stop)

In [ ]:
show_tables(conn)

In [ ]:
drop_temporary_tables(conn)

In [ ]:
show_tables(conn)

In [ ]:
cur = conn.cursor()
cur.execute("INSERT INTO scoring_key (type) VALUES ('Automatic');")
conn.commit()
key_id = cur.lastrowid

In [ ]:
print_query("SELECT * FROM scoring_key")

In [ ]:
cur = conn.cursor()
cur.execute("INSERT INTO scoring_revision (name, is_deleted, version) VALUES ('Gandalf', 0, 0);")
conn.commit()
scoring_revision_id = cur.lastrowid

In [ ]:
print_query("SELECT * FROM scoring_revision")

In [ ]:
print(scoring_revision_id,key_id)

In [ ]:
cur = conn.cursor()
cur.execute(f"INSERT INTO scoring_revision_to_key (revision_id, key_id) VALUES ({scoring_revision_id}, {key_id});")
conn.commit()

In [ ]:
print_query("SELECT * FROM scoring_revision_to_key")

In [ ]:
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS internal_property;")
conn.commit()

In [ ]:
conn.close()